In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [3]:
# 필요 라이브러리 임포트
import os
import json
import re
from pydantic import BaseModel, Field
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

In [4]:
######## states 정의 ########
class InputState(TypedDict):
    start_input: str

class OverallState(TypedDict):
    user_input: str
    messages: Annotated[list, add_messages]
    Stock_Value_dict: dict
    query_list: List[str]

class EndState(TypedDict):
    messages: Annotated[list, add_messages]
    query_list: list

In [5]:
# 사용자 입력 노드 정의
def user_input_node(state: InputState):
    print("================================= calculation stock =================================")
    print("주식 가치를 분석합니다. 궁금하신 주식명을 말씀해주세요.")
    user_input = input("User: ")

    return {
        "user_input": user_input,
        "messages": [("user", user_input)],
        "Stock_Value_dict": {},  # 나중에 여기에 데이터 채워짐
    }

In [6]:
def search_query_generation_node(state: OverallState):
    user_input = state["user_input"]
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 금융 정보를 찾기 위한 검색 쿼리를 생성하는 전문가입니다."),
        ("human", '''
사용자 입력: "{user_input}"을 참고하여 아래 내용에 대해 검색 쿼리를 만들어주세요.
  "당기순이익",
  "발행주식수",
  "현재 주가",
  "자본총계",
  "자유현금흐름",
  "영업이익",
  "가중평균자본비용(WACC)",
  "예상미래현금흐름",
  "성장률",
  "주당배당금",
  "기타 수익 관련 정보"
출력 형식은 반드시 JSON 배열입니다. 
''')
    ])

    messages = prompt.format_messages(user_input=user_input)
    response = llm.invoke(messages)

    try:
        cleaned = re.sub(r"^```(?:json)?\n|\n```$", "", response.content.strip())
        query_list = json.loads(cleaned)
    except Exception as e:
        print(f"❗ 검색 쿼리 파싱 실패: {e}")
        query_list = []

    return {
        **state,
        "query_list": query_list
    }

In [7]:
def query_refine_node(state: OverallState) -> OverallState:
    original_queries = state.get("query_list", [])
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

    # 🔹 few-shot 예시
    examples = [
        {
            "input": ["삼성전자 당기순이익", "삼성전자 발행주식수"],
            "output": ["삼성전자 2025년 연결 기준 당기순이익", "삼성전자 2025년 발행주식수"]
        },
        {
            "input": ["LG에너지솔루션 영업이익", "LG에너지솔루션 성장률"],
            "output": ["LG에너지솔루션 2025년 연결 기준 영업이익", "LG에너지솔루션 2025년 예상 성장률"]
        }
    ]

    few_shot_format = "\n\n".join([
        f"입력: {ex['input']}\n보정 결과: {ex['output']}"
        for ex in examples
    ])

    refine_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
당신은 전자공시시스템(DART) 문서에 적합한 검색어를 보정하는 전문가입니다.

다음 원본 검색어 목록을 참고하여 DART 문서에서 실제로 검색이 잘 되도록 쿼리를 보정해주세요.

### 보정 규칙:
- 연결/개별 기준이 있다면 그대로 유지
- 2025년 같은 연도 정보는 **붙이지 마세요**
- "현금흐름" → "현금흐름표", "배당금" → "배당금 지급 계획" 등 보고서 용어에 맞게 고치세요
- 보정된 쿼리는 문서에서 바로 검색이 가능해야 합니다.

출력 형식은 반드시 JSON 배열입니다.

아래는 예시입니다:
{few_shot_format}
"""),
        ("human", """
원본 쿼리 목록:
{query_list}
""")
    ])

    messages = refine_prompt.format_messages(query_list=json.dumps(original_queries, ensure_ascii=False))
    response = llm.invoke(messages)

    try:
        cleaned = re.sub(r"^```(?:json)?\n|\n```$", "", response.content.strip())
        refined_queries = json.loads(cleaned)
    except Exception as e:
        print(f"❗ 쿼리 보정 실패: {e}")
        refined_queries = original_queries

    print("\n🔁 쿼리 보정 결과:")
    for o, r in zip(original_queries, refined_queries):
        print(f"- {o} → {r}")

    return {
        **state,
        "query_list": refined_queries,
        "previous_query": original_queries,
    }


In [8]:
######## nodes.py ########
# --- 문서 로드 및 전처리 ---
loader = PyMuPDFLoader("stock_report/[삼성전자]분기보고서(2024.11.14).pdf")
docs = loader.load()

## : 문서 분할(Split Documents) <-----------추후 문서 제목 단위 분할로 변경 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
split_documents = text_splitter.split_documents(docs)

# 중복 제거
unique_documents = []
seen_contents = set()

for doc in split_documents:
    content = doc.page_content.strip()
    if content not in seen_contents:
        seen_contents.add(content)
        unique_documents.append(doc)

print(f"원본 문서 수: {len(split_documents)}")
print(f"중복 제거 후 문서 수: {len(unique_documents)}")

## 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 벡터스토어 생성
vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings, persist_directory="stock_report/chroma_db")

# 5. 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

원본 문서 수: 404
중복 제거 후 문서 수: 403


In [9]:
key_mapping = {
    "당기순이익": "Net_Income",
    "발행주식수": "Shares_Outstanding",
    "현재 주가": "Stock_Price",
    "자본총계": "Shareholders_equity",
    "자유현금흐름": "Free_cash_flow",
    "영업이익": "Operating_income",
    "가중평균자본비용": "WACC",
    "예상미래현금흐름": "Projected_future_cash_flows",
    "성장률": "Growth_Rate",
    "주당배당금": "Dividend_per_share",
    "기타 수익 관련 정보": "Other_return_related_information",
}

summary_prompt = ChatPromptTemplate.from_messages([
    ("system", """
당신은 기업의 재무 정보를 요약하는 분석 전문가입니다.
다음 문서를 읽고, 아래 항목에 대해 **정확한 값과 단위**를 JSON 형식으로 요약해 주세요.

반드시 아래 형식으로 응답하세요 (JSON):
{{
  "당기순이익": "...",
  "발행주식수": "...",
  "현재 주가": "...",
  "자본총계": "...",
  "자유현금흐름": "...",
  "영업이익": "...",
  "가중평균자본비용(WACC)": "...",
  "예상미래현금흐름": "...",
  "성장률": "...",
  "주당배당금": "...",
  "기타 수익 관련 정보": "..."
}}

값을 찾을 수 없거나 문서에 명확히 없으면 "없음"이라고 표기하세요.
"""),
    ("human", "{context}")
])

def summarize_context(context: str, llm) -> dict:
    messages = summary_prompt.format_messages(context=context)
    response = llm.invoke(messages)

    try:
        raw = re.sub(r"^```(?:json)?\n|\n```$", "", response.content.strip())
        parsed = json.loads(raw)
        return {key_mapping.get(k, k): v for k, v in parsed.items()}
    except Exception as e:
        print(f"❗ 요약 파싱 실패: {e}")
        return {}

In [10]:
def merge(parsed: dict, stock_data: dict):
    for k, v in parsed.items():
        if k not in stock_data or stock_data[k] == "없음":
            stock_data[k] = v

def query_loop_node(state: OverallState) -> OverallState:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tavily = TavilySearchResults(max_results=3)

    stock_data = {}
    fallback_queries = []
    key_fields_to_check = ["Net_Income", "Operating_income", "Free_cash_flow"]

    for query in state["query_list"]:
        print(f"\n🔍 [RAG 검색] {query}")

        try:
            docs = list({doc.page_content.strip() for doc in retriever.invoke(query)})
            context = "\n\n".join(docs)[:3000]
        except Exception as e:
            print(f"❗ RAG 검색 중 오류 발생: {e}")
            context = ""

        parsed = summarize_context(context, llm)

        if all(parsed.get(k, "없음") == "없음" for k in key_fields_to_check):
            print("⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도")
            fallback_queries.append(query)

            try:
                web_result = tavily.invoke({"query": query})

                # ✅ 웹 결과가 list인지 확인
                if isinstance(web_result, list):
                    web_context = "\n\n".join([doc.get("content", "") for doc in web_result])[:3000]
                else:
                    web_context = "\n\n".join([doc.get("content", "") for doc in web_result.get("documents", [])])[:3000]

                if web_context.strip():
                    parsed = summarize_context(web_context, llm)

            except Exception as e:
                print(f"❗ 웹 검색 중 오류 발생: {e}")

        # regardless of source (RAG or web), merge
        merge(parsed, stock_data)

    # 누락된 항목 "없음" 채우기
    for key in key_mapping.values():
        if key not in stock_data:
            stock_data[key] = "없음"

    return {
        **state,
        "Stock_Value_dict": stock_data,
        "fallback_queries": fallback_queries,
        "messages": state["messages"] + [AIMessage(content="✅ 핵심 키 기준 RAG + 웹 fallback 완료")],
    }


In [11]:
def web_fallback_node(state: OverallState) -> OverallState:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    web = TavilySearchResults(max_results=3)
    stock_data = state["Stock_Value_dict"]
    fallback_queries = state.get("fallback_queries", [])

    for query in fallback_queries:
        print(f"🌐 [웹 검색 fallback] {query}")
        web_result = web.invoke({"query": query})
        web_context = "\n\n".join([d["content"] for d in web_result["documents"]])[:3000]
        parsed = summarize_context(web_context, llm)
        merge(parsed, stock_data)

    return {
        "Stock_Value_dict": stock_data,
        "messages": state["messages"] + [AIMessage(content="🌐 웹 검색 보완 완료")],
    }

In [12]:
def stock_valuation_node(state: OverallState) -> OverallState:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    financials = state.get("Stock_Value_dict", {})
    user_input = state.get("user_input", "")

    valuation_prompt = ChatPromptTemplate.from_messages([
        ("system", """
당신은 기업의 재무 데이터를 바탕으로 가치를 평가하는 금융 전문가입니다.

다음 재무 데이터를 기반으로 **삼성전자의 주식 가치를 추정**해 주세요.
- 가능하면 **자유현금흐름 할인법(DCF)** 모델을 사용하고, 
- **PER/PBR** 기반의 보완적 계산도 함께 진행해 주세요.

출력은 다음 형식을 따릅니다:

---
1. 사용된 주요 재무 지표 요약
2. [DCF] 방식에 의한 주당 적정 주가 추정
3. [PER/PBR] 방식에 의한 주당 적정 주가 추정
4. 최종 판단 및 투자자 참고 사항

데이터:
{financials}
"""),
        ("human", f"{user_input}의 주식 가치를 평가해줘.")
    ])

    messages = valuation_prompt.format_messages(financials=json.dumps(financials, ensure_ascii=False))
    response = llm.invoke(messages)

    print("\n📈 주식 가치 평가 결과:")
    print(response.content.strip())

    return {
        **state,
        "valuation_result": response.content.strip(),
        "messages": state["messages"] + [AIMessage(content="📈 주식 가치 평가 완료")]
    }


In [13]:
# 메모리 및 상태 그래프 초기화
memory = MemorySaver()
graph_builder = StateGraph(OverallState, input=InputState, output=EndState)

# 노드 등록
graph_builder.add_node("User Input", user_input_node)
graph_builder.add_node("Search Query Generation", search_query_generation_node)
graph_builder.add_node("Query Refinement", query_refine_node)
graph_builder.add_node("Query Loop (RAG + Web)", query_loop_node)
graph_builder.add_node("Web Fallback", web_fallback_node)
graph_builder.add_node("Stock Valuation", stock_valuation_node)

# 엣지 연결
graph_builder.set_entry_point("User Input")  # 시작 노드
graph_builder.add_edge("User Input", "Search Query Generation")
graph_builder.add_edge("Search Query Generation", "Query Refinement")
graph_builder.add_edge("Query Refinement", "Query Loop (RAG + Web)")
graph_builder.add_edge("Query Loop (RAG + Web)", "Stock Valuation")
graph_builder.add_edge("Stock Valuation", END)

# 그래프 컴파일
graph = graph_builder.compile()

In [14]:
# 초기 상태
initial_state = {
    "start_input": "",
}

# 그래프 실행
final_state = graph.invoke(initial_state)

================================= calculation stock =================================
주식 가치를 분석합니다. 궁금하신 주식명을 말씀해주세요.


User:  삼성전자



🔁 쿼리 보정 결과:
- 삼성전자 당기순이익 → 삼성전자 연결 기준 당기순이익
- 삼성전자 발행주식수 → 삼성전자 발행주식수
- 삼성전자 현재 주가 → 삼성전자 현재 주가
- 삼성전자 자본총계 → 삼성전자 자본총계
- 삼성전자 자유현금흐름 → 삼성전자 자유현금흐름표
- 삼성전자 영업이익 → 삼성전자 영업이익
- 삼성전자 가중평균자본비용(WACC) → 삼성전자 가중평균자본비용(WACC)
- 삼성전자 예상미래현금흐름 → 삼성전자 예상 미래 현금흐름
- 삼성전자 성장률 → 삼성전자 성장률
- 삼성전자 주당배당금 → 삼성전자 주당 배당금 지급 계획
- 삼성전자 기타 수익 관련 정보 → 삼성전자 기타 수익 관련 정보

🔍 [RAG 검색] 삼성전자 연결 기준 당기순이익

🔍 [RAG 검색] 삼성전자 발행주식수
⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도

🔍 [RAG 검색] 삼성전자 현재 주가
⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도

🔍 [RAG 검색] 삼성전자 자본총계
⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도

🔍 [RAG 검색] 삼성전자 자유현금흐름표
⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도

🔍 [RAG 검색] 삼성전자 영업이익

🔍 [RAG 검색] 삼성전자 가중평균자본비용(WACC)
⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도

🔍 [RAG 검색] 삼성전자 예상 미래 현금흐름
⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도

🔍 [RAG 검색] 삼성전자 성장률
⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도

🔍 [RAG 검색] 삼성전자 주당 배당금 지급 계획
⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도

🔍 [RAG 검색] 삼성전자 기타 수익 관련 정보
⚠️ LLM이 핵심 값을 추출 못함 → 웹 검색 시도

📈 주식 가치 평가 결과:
---
1. 사용된 주요 재무 지표 요약
   - 순이익(Net Income): 2,475억원
   - 발행주식수(Shares Outstanding): 